<a href="https://colab.research.google.com/github/ahmad811/nlp/blob/main/commands_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pickle

import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
with open('/content/gdrive/MyDrive/SIEMESN/Innovation/nlp/cmd2id.pickle', 'rb') as f:
    cmd2idMap = pickle.load(f)
with open('/content/gdrive/MyDrive/SIEMESN/Innovation/nlp/id2cmd.pickle', 'rb') as f:
    id2cmdMap = pickle.load(f)

with open('/content/gdrive/MyDrive/SIEMESN/Innovation/nlp/book_np.pickle', 'rb') as f:
    corpus = pickle.load(f)

unique_cmds = cmd2idMap.keys()

corpus_len = len(corpus)
vocab_len = len(unique_cmds)

print(corpus_len)
print(vocab_len)


211292812
809


In [4]:
seq_len = 20

X=[]
y=[]

first_chunk=500_000

for i in range(seq_len+1, first_chunk):#corpus_len):
    seq = corpus[i-seq_len-1:i]
    X.append(seq[:-1]) #all but last one
    y.append(seq[-1]) #last one


In [5]:
y = to_categorical(y, num_classes=vocab_len, dtype=bool)

print(y.shape)

(499979, 809)


In [6]:
X = np.array(X, dtype='int16')
#X = to_categorical(X, num_classes=vocab_len, dtype=bool)

print(X.shape)
x_dim = (10,vocab_len)


(499979, 20)


In [53]:
#a=X.reshape(X.shape[0],10,-1)
#print(a.shape)
#print(a[0])

(249989, 10, 1)
[[743]
 [433]
 [ 10]
 [433]
 [ 10]
 [755]
 [755]
 [433]
 [ 10]
 [448]]


In [7]:
#Build the model
model = Sequential()

model.add(Embedding(input_dim=vocab_len, output_dim=7, input_length=seq_len))

model.add(LSTM(2048,  return_sequences=True))
model.add(LSTM(1024, return_sequences=True))
#model.add(LSTM(1024, return_sequences=True))
model.add(LSTM(1024))

model.add(Dense(1024, activation='relu'))
model.add(Dense(vocab_len, activation='softmax'))

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 7)             5663      
_________________________________________________________________
lstm (LSTM)                  (None, 20, 2048)          16842752  
_________________________________________________________________
lstm_1 (LSTM)                (None, 20, 1024)          12587008  
_________________________________________________________________
lstm_2 (LSTM)                (None, 1024)              8392704   
_________________________________________________________________
dense (Dense)                (None, 1024)              1049600   
_________________________________________________________________
dense_1 (Dense)              (None, 809)               829225    
Total params: 39,706,952
Trainable params: 39,706,952
Non-trainable params: 0
____________________________________________

In [8]:
#Callbacks
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard

checkpoint = ModelCheckpoint("nextcommand.h5", monitor='loss', verbose=1, save_best_only=True, mode='auto')



In [ ]:

#Compile and Fit
model.compile (loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'] )
history = model.fit(X, y, batch_size=512, epochs=100, callbacks=[checkpoint], shuffle=False)


Epoch 1/100
463/977 [=============>................] - ETA: 9:56 - loss: 3.5029 - accuracy: 0.2328

In [ ]:
from matplotlib import pyplot
pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'], loc='upper right')
pyplot.show()

In [ ]:
model.save('/content/gdrive/MyDrive/SIEMESN/Innovation/nlp/model')
model.save_weights('/content/gdrive/MyDrive/SIEMESN/Innovation/nlp/model.weights')

# New Section